## Import der benötigten Bibliotheken

In [26]:
import pprint
import csv
import pandas as pd
from pymongo import MongoClient

## Konvertierung der .txt-Datenquellen in .csv-Dateien
Muss nur beim ersten Mal ausgeführt werden

In [ ]:
sentences_csv = pd.read_csv ('deu_news_2015_3M-sentences.txt', header = None, delimiter='\t')
sentences_csv.to_csv ('sentences_de.csv', index=None)

In [ ]:
stopwords_csv = pd.read_csv ('stopwords_deutsch.txt', header = None, delimiter='\t')
stopwords_csv.to_csv ('stopwords_de.csv', index=None)

## Verbindung mit mongoDB

In [27]:
connection_string = "mongodb://localhost:27017"
db                = MongoClient(connection_string)
sentences         = db.dw.sentences
stopwords         = db.dw.stopwords

## Sentences und Stopwords der Database in mongoDB hinzufügen

`read_csv()` liest eine .csv-Datei ein und gibt ein Dictionary zurück, dass dann mit `save_to_mongo()` in einer Collection gespeichert wird. Dafür kann zunächst die Collection gelöscht werden.

In [28]:
def read_csv(path, columns):
    reader = open(path, 'r', encoding='utf-8')
    return csv.DictReader(reader, columns)   

In [29]:
def save_to_mongo(collection, path, columns, drop_collection):
    if drop_collection:
        collection.drop()
    data = read_csv(path, columns)
    result = collection.insert_many(data)
    print('%d rows are saved to "%s" collection successfully!' % (len(result.inserted_ids), collection.name))

In [11]:
save_to_mongo(sentences, 'sentences_de.csv', ('id','sentence'), True)

2646184 rows are saved to "sentences" collection successfully!


In [12]:
save_to_mongo(stopwords, 'stopwords_de.csv', ('s'), True)

604 rows are saved to "stopwords" collection successfully!


Wenn ich die als Column-Name gleich `stopwords` genommen habe, dann hat er daraus eine Spalte `s`, eine `t`, eine `o` und so weiter gemacht...Deswegen hier mit der späteren Umbenennung.

In [ ]:
stopwords.update_many( {}, { '$rename': { "s": "stopword" } } )

### Blödsinn von Lea:

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     D:\Users\BKU\LeaPellikan\AppData(Roaming)\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [30]:
from nltk import word_tokenize
from nltk.corpus import stopwords

In [31]:
stop = set(stopwords.words('german'))
sentence = "das ist das Haus vom Nikolaus."

In [46]:
nostop = {i for i in sentence.lower().split() if i not in stop}

#convert set to dict
ini_set = {"id", "sentence"} 
nostop_dict = dict.fromkeys(ini_set, 12345)
print(type(nostop_dict))
print(nostop_dict)

#insert dict to database
result = sentences.insert_many(nostop_dict)
print(nostop)
print(result)

<class 'dict'>
{'sentence': 12345, 'id': 12345}


TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

Adding punctuation to the deleted words:

In [33]:
stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])